In [78]:
!pip install undetected-chromedriver selenium pandas webdriver-manager



In [79]:
import time, pickle, os
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException

In [80]:
#########################################
# LOGIN DETAILS
#########################################
EMAIL = "khanfaizanurrahim@gmail.com"
PASSWORD = "F@izan_Khan17"

JOB_KEYWORD = "Cyber Security"
JOB_LOCATION = "United States"


In [81]:
# ------------------------------
# 1. Setup driver
# ------------------------------
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 12)

COOKIE_FILE = "linkedin_cookies.pkl"

def save_cookies():
    pickle.dump(driver.get_cookies(), open(COOKIE_FILE, "wb"))

def load_cookies():
    if os.path.exists(COOKIE_FILE):
        driver.get("https://www.linkedin.com/")
        for cookie in pickle.load(open(COOKIE_FILE, "rb")):
            driver.add_cookie(cookie)
        driver.refresh()
        print("Cookies loaded, session restored.")
        return True
    return False

In [82]:
# ------------------------------
# 2. Login function
# ------------------------------
def linkedin_login(email=None, password=None):
    driver.get("https://www.linkedin.com/login")
    if email and password:
        wait.until(EC.presence_of_element_located((By.ID, "username"))).send_keys(email)
        driver.find_element(By.ID, "password").send_keys(password)
        driver.find_element(By.CSS_SELECTOR, "button[type='submit']").click()
    print("Login attempted. Solve CAPTCHA manually if prompted.")
    time.sleep(10)

In [83]:
# ------------------------------
# 3. Safe text helper
# ------------------------------
def safe_text(selector, by=By.CSS_SELECTOR, wait_time=5):
    try:
        element = WebDriverWait(driver, wait_time).until(
            EC.presence_of_element_located((by, selector))
        )
        return element.text.strip()
    except Exception:
        return None

In [84]:
# ------------------------------
# 5. Get job cards
# ------------------------------

def get_job_cards():
    selectors = [
        "li.jobs-search-results__list-item",
        "div.job-card-container",
        "div.jobs-search-results__list-item"
    ]
    for sel in selectors:
        cards = driver.find_elements(By.CSS_SELECTOR, sel)
        if cards:
            return cards
    return []

In [85]:
# ------------------------------
# 4. Scroll job panel
# ------------------------------
def scroll_page(max_scrolls=8, pause=1.5):
    last_height = driver.execute_script("return document.body.scrollHeight;")
    for _ in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new_height = driver.execute_script("return document.body.scrollHeight;")
        if new_height == last_height:
            break
        last_height = new_height

In [86]:
# ------------------------------
# 6. Scrape jobs
# ------------------------------
def scrape_jobs(search_url, max_pages=5):
    jobs_data = []
    driver.get(search_url)
    time.sleep(5)

    for page in range(max_pages):
        print(f"Scraping page {page+1}...")
        scroll_page()

        job_cards = get_job_cards()
        print(f"Found {len(job_cards)} jobs.")

        for i, job in enumerate(job_cards):
            try:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", job)
                job.click()
                time.sleep(2)

                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.jobs-details__main-content"))
                )

                title = safe_text("h1.top-card-layout__title")
                company = safe_text("div.jobs-unified-top-card__company-name")
                location = safe_text("div.jobs-unified-top-card__bullet")
                description = safe_text("div.show-more-less-html__markup")
                link = driver.current_url

                jobs_data.append({
                    "Job Title": title,
                    "Company": company,
                    "Location": location,
                    "Description": description,
                    "Link": link
                })
            except Exception as e:
                print(f"Error scraping job {i}: {e}")

        # Next page if available
        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, "button[aria-label='Page next']")
            if next_btn.is_enabled():
                next_btn.click()
                time.sleep(5)
            else:
                break
        except NoSuchElementException:
            break

    return pd.DataFrame(jobs_data)


In [87]:
# ------------------------------
# 7. Usage
# ------------------------------
EMAIL = "khanfaizanurrahim@gmail.com"
PASSWORD = "F@izan_Khan17"

options = uc.ChromeOptions()
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 10)

linkedin_login(EMAIL, PASSWORD)

search_url = "https://www.linkedin.com/jobs/search/?keywords=Cyber%20Security&location=United%20States"
df = scrape_jobs(search_url, max_pages=5)

print(f"Total jobs collected: {len(df)}")
print(df.head())


Login attempted. Solve CAPTCHA manually if prompted.
Scraping page 1...
Found 7 jobs.
Total jobs collected: 7
  Job Title Company Location Description  \
0      None    None     None        None   
1      None    None     None        None   
2      None    None     None        None   
3      None    None     None        None   
4      None    None     None        None   

                                                Link  
0  https://www.linkedin.com/jobs/search/?currentJ...  
1  https://www.linkedin.com/jobs/search/?currentJ...  
2  https://www.linkedin.com/jobs/search/?currentJ...  
3  https://www.linkedin.com/jobs/search/?currentJ...  
4  https://www.linkedin.com/jobs/search/?currentJ...  
